In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Create results directory
os.makedirs('/content/results', exist_ok=True)

# Load the Iris dataset
iris = sns.load_dataset('iris')

# Create a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=iris, x='sepal_length', y='sepal_width', hue='species')
plt.title('Sepal Length vs Sepal Width for Iris Species')
plt.xlabel('Sepal Length (cm)')
plt.ylabel('Sepal Width (cm)')
plt.savefig('/content/results/iris_scatter_plot.png')
plt.show()

In [ ]:
# --- Set GitHub credentials manually ---
import os

# Example (replace with your values):
# os.environ["GITHUB_TOKEN"] = "ghp_your_personal_access_token_here"
# os.environ["GITHUB_USER"] = "your_github_username"
# os.environ["GITHUB_REPO"] = "your_repository_name"
# os.environ["BRANCH"] = "main"  # optional


In [ ]:
import os
import subprocess
import shutil
from pathlib import Path

token = os.environ.get("GITHUB_TOKEN")
user = os.environ.get("GITHUB_USER")
repo = os.environ.get("GITHUB_REPO")
branch = os.environ.get("BRANCH", "main")
notebook_name = "generated_notebook.ipynb"  # notebook title used for folder

if not all([token, user, repo]):
    raise ValueError("Please set GITHUB_TOKEN, GITHUB_USER, and GITHUB_REPO before running this cell.")

repo_url = f"https://{token}@github.com/{user}/{repo}.git"
repo_dir = "/content/temp_repo"
results_dir = "/content/results"

if not os.path.exists(results_dir) or not any(Path(results_dir).iterdir()):
    raise ValueError(f"No files found in {results_dir}. Make sure you saved your plots or outputs there.")

if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)

print("📥 Cloning repository...")
subprocess.run(["git", "clone", repo_url, repo_dir], check=True)

subprocess.run(["git", "-C", repo_dir, "config", "user.email", "colab@example.com"], check=True)
subprocess.run(["git", "-C", repo_dir, "config", "user.name", "Colab User"], check=True)

base_name = Path(notebook_name).stem
dest_dir = os.path.join(repo_dir, base_name)
os.makedirs(dest_dir, exist_ok=True)

for item in Path(results_dir).iterdir():
    shutil.copy(item, dest_dir)

print(f"📂 Copied results to {dest_dir}")

subprocess.run(["git", "-C", repo_dir, "add", "."], check=True)
diff_proc = subprocess.run(["git", "-C", repo_dir, "diff", "--cached", "--quiet"])
if diff_proc.returncode != 0:
    subprocess.run(["git", "-C", repo_dir, "commit", "-m", f"Add results from generated_notebook.ipynb"], check=True)
    subprocess.run(["git", "-C", repo_dir, "push", "origin", branch], check=True)
    print(f"✅ Results from 'generated_notebook.ipynb' pushed successfully to GitHub!")
else:
    print("⚠️ No new files to commit. Nothing was pushed.")
